# Project Finance

**Objetivo:** Analisar os dados de ativos da IBOVESPA e criar um modelo de times series que possa prever a curto ou médio prazo o preço das ações com base em dados históricos da empresa, tais como:

1. Valores dos ativos historicamente.
2. Resultados trimestrais.
3. Notícias.

In [2]:
import pandas as pd
import numpy as np
import dash
import os